In [5]:
import numpy as np
from data_loading_and_preprocessing import *
from find_hyperparameter import *
from Augmenting_data import *
from implementations import *
from proj1_helpers import create_csv_submission


In [21]:
path =  "C:/Users/jurri/OneDrive/Documenten/University/Exchange/ML/train"
X_train, ids = load_data_features(path +"/train.csv")
y_train = load_results(path +"/train.csv")

"""Use this cell if y_test is NOT known"""
X_test, ids_test = load_data_features(path +"/test.csv")

X_train, X_test_pro, y_train, _, ids_tr_group, ids_test_group = process_data(X_train, 
                                                    X_test, y_train, None, ids_train, ids_test)

#parameters to be changed in this fucntion
#test fro sepcific degrees for each subset
degrees = [np.linspace(3,9,7, dtype = int), np.linspace(6, 14, 9, dtype = int), np.linspace(9, 17, 8, dtype = int)]

#test for specific lambdas for each subset
lambdas_base = np.array([0])
lambdas = [np.append(np.logspace(-13,-9,6), lambdas_base),
          np.append(np.logspace(-13,-5,9), lambdas_base),
          np.append(np.logspace(-13, -9,6), lambdas_base)]

#test for both methods
methods = ["Ridge_regression", "Penalized_logistic"]

best_parameter_per_set, losses_per_set = find_parameters(X_train, y_train, degrees, lambdas, methods)
y_result = []
for i in range(3):
    method = best_parameter_per_set[i][0]
    degree = int(best_parameter_per_set[i][1])
    lambda_ = best_parameter_per_set[i][2]
    
    X_train_ex = add_features(X_train[i], degree = degree)
    X_test_pro_ex = add_features(X_test_pro[i], degree = degree)
    
    if method == "Ridge_regression":
        w = ridge_regression(y_train[i], X_train_ex, lambda_)
        y_result.append(X_test_pro_ex @ w)
    elif method == "Penalized_logistic":
        # needs bit more
        _ , w = logistic_regression_penalized_gradient_descent(y_train[i], X_train_ex, lambda_,
                                                               gamma = 10**-10, max_iter = 3000)
        y_result.append(X_test_pro_ex @ w)
        

y_final = stitch_solution(X_test, y_result, ids_test_group, ids_test)
y_final[y_final < 0.5] = 0
y_final[y_final >= 0.5] = 1

name = "final_submission.csv"
create_csv_submission(ids_test, y_final, name)



NameError: name 'X_train' is not defined

In [ ]:
path =  "C:/Users/jurri/OneDrive/Documenten/University/Exchange/ML/train"
X, ids = load_data_features(path +"/train.csv")
y = load_results(path +"/train.csv")

"""Use this cell if the training data will be split in a test and a train set"""

X_train, y_train, ids_train = X[:int(0.8*len(X)),:], y[:int(0.8*len(X))], ids[:int(0.8*len(X))]
X_test, y_test, ids_test = X[int(0.8*len(X)):,:], y[int(0.8*len(X)):], ids[int(0.8*len(X)):]

X_train, X_test_pro, y_train, y_test_pro, ids_tr_group, ids_test_group = process_data(X_train, 
                                                    X_test, y_train, y_test, ids_train, ids_test)

#parameters to be changed in this fucntion
#test fro sepcific degrees for each subset
degrees = [np.linspace(3,9,7, dtype = int), np.linspace(6, 14, 9, dtype = int), np.linspace(9, 17, 8, dtype = int)]

#test for specific lambdas for each subset
lambdas_base = np.array([0])
lambdas = [np.append(np.logspace(-13,-9,6), lambdas_base),
          np.append(np.logspace(-13,-5,9), lambdas_base),
          np.append(np.logspace(-13, -9,6), lambdas_base)]

#test for both methods
methods = ["Ridge_regression", "Penalized_logistic"]

best_parameter_per_set, losses_per_set = find_parameters(X_train, y_train, degrees, lambdas, methods)

y_result = []
for i in range(3):
    lambda_ = best_parameter_per_set[i][0,0]
    degree = int(best_parameter_per_set[i][0,1])
    X_train_ex = add_features(X_train[i], degree = degree)
    X_test_pro_ex = add_features(X_test_pro[i], degree = degree)
    show_result_logistic(X_train_ex, y_train[i], X_test_pro_ex, y_test_pro[i], lambda_, gamma = 0.000001)
    
    w = ridge_regression(y_train[i], X_train_ex, lambda_)
    y_result.append(X_test_pro_ex @ w)

y_final = stitch_solution(X_test, y_result, ids_test_group, ids_test)
y_test = np.reshape(y_test, (len(y_test),1))
y_final = np.array(y_final)
quantify_result(y_final, y_test)


